In [2]:
import pandas as pd

In [3]:
dfs = {
    "Liverpool": "https://fbref.com/en/squads/822bd0ba/2024-2025/matchlogs/c9/schedule/Liverpool-Scores-and-Fixtures-Premier-League",
    "Arsenal": "https://fbref.com/en/squads/18bb7c10/2024-2025/matchlogs/c9/schedule/Arsenal-Scores-and-Fixtures-Premier-League",
    "Nottingham Forest": "https://fbref.com/en/squads/e4a775cb/2024-2025/matchlogs/all_comps/schedule/Nottingham-Forest-Scores-and-Fixtures-Premier-League",
    "Chelsea": "https://fbref.com/en/squads/cff3d9bb/2024-2025/matchlogs/all_comps/schedule/Chelsea-Scores-and-Fixtures-Premier-League",
    "Manchester City": "https://fbref.com/en/squads/b8fd03ef/2024-2025/matchlogs/all_comps/schedule/Manchester-City-Scores-and-Fixtures-Premier-League",
    "Newcastle United": "https://fbref.com/en/squads/b2b47a98/2024-2025/matchlogs/c9/schedule/Newcastle-United-Scores-and-Fixtures-Premier-League",
    "Bournemouth": "https://fbref.com/en/squads/4ba7cbea/2024-2025/matchlogs/c9/schedule/Bournemouth-Scores-and-Fixtures-Premier-League",
    "Aston Villa": "https://fbref.com/en/squads/8602292d/2024-2025/matchlogs/c9/schedule/Aston-Villa-Scores-and-Fixtures-Premier-League",
    "Fulham": "https://fbref.com/en/squads/fd962109/2024-2025/matchlogs/c9/schedule/Fulham-Scores-and-Fixtures-Premier-League",
    "Brighton": "https://fbref.com/en/squads/d07537b9/2024-2025/matchlogs/c9/schedule/Brighton-and-Hove-Albion-Scores-and-Fixtures-Premier-League",
    "Brentford": "https://fbref.com/en/squads/cd051869/2024-2025/matchlogs/c9/schedule/Brentford-Scores-and-Fixtures-Premier-League",
    "Crystal Palace": "https://fbref.com/en/squads/47c64c55/2024-2025/matchlogs/c9/schedule/Crystal-Palace-Scores-and-Fixtures-Premier-League",
    "Manchester United": "https://fbref.com/en/squads/19538871/2024-2025/matchlogs/c9/schedule/Manchester-United-Scores-and-Fixtures-Premier-League",
    "Spurs": "https://fbref.com/en/squads/361ca564/2024-2025/matchlogs/c9/schedule/Tottenham-Hotspur-Scores-and-Fixtures-Premier-League",
    "West Ham": "https://fbref.com/en/squads/7c21e445/2024-2025/matchlogs/c9/schedule/West-Ham-United-Scores-and-Fixtures-Premier-League",
    "Everton": "https://fbref.com/en/squads/d3fd31cc/2024-2025/matchlogs/c9/schedule/Everton-Scores-and-Fixtures-Premier-League",
    "Wolves": "https://fbref.com/en/squads/8cec06e1/2024-2025/matchlogs/c9/schedule/Wolverhampton-Wanderers-Scores-and-Fixtures-Premier-League",
    "Leicester": "https://fbref.com/en/squads/a2d435b3/2024-2025/matchlogs/c9/schedule/Leicester-City-Scores-and-Fixtures-Premier-League",
    "Ipswich": "https://fbref.com/en/squads/b74092de/2024-2025/matchlogs/c9/schedule/Ipswich-Town-Scores-and-Fixtures-Premier-League",
    "Southampton": "https://fbref.com/en/squads/33c895d4/2024-2025/matchlogs/c9/schedule/Southampton-Scores-and-Fixtures-Premier-League"
}

In [31]:
# Transform each URL into a dataframe
dfs = {team: pd.read_html(url, attrs={"id":"matchlogs_for"})[0] for team, url in dfs.items()}

In [32]:
# Remove the rows where result is NaN
dfs = {team: df[df['Result'].notna()] for team, df in dfs.items()}

In [ ]:
dfs['Liverpool']

In [58]:
# First, let's create a function to determine if the team with more possession won
def analyze_possession_result(row):
    # If possession is greater than 50%, team had more possession
    had_more_possession = row['Poss'] > 50
    
    # Determine if team won
    won_game = row['Result'] == 'W'
    drew_game = row['Result'] == 'D'
    
    # Return result
    if drew_game:
        return 'Draw'
    return 'Won' if had_more_possession == won_game else 'Lost'

# Create a list to store all possession-result pairs
possession_results = []
total_matches = 0
matches_with_possession = 0

# Analyze each team's matches
for team, df in dfs.items():
    total_matches += len(df)
    if 'Poss' in df.columns:  # Make sure possession data exists
        valid_matches = df[df['Poss'].notna()]  # Only include matches with possession data
        matches_with_possession += len(valid_matches)
        valid_matches['possession_result'] = valid_matches.apply(analyze_possession_result, axis=1)
        possession_results.extend(valid_matches['possession_result'].tolist())

/var/folders/pw/3lgbkgk149n6t5s78s_4n0rc0000gn/T/ipykernel_33942/886213565.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_matches['possession_result'] = valid_matches.apply(analyze_possession_result, axis=1)


In [ ]:
# Plot the dataframe
import matplotlib.pyplot as plt

# Create a DataFrame with the results and calculate percentages
results_df = pd.DataFrame(possession_results, columns=['Outcome'])
results_summary = results_df['Outcome'].value_counts()
results_percentages = results_df['Outcome'].value_counts(normalize=True) * 100

# Create the bar plot
plt.style.use('default')
plt.figure(figsize=(10, 6))
bars = plt.bar(results_percentages.index, results_percentages.values)

# Add percentage labels on top of each bar
for bar in bars:
    height = bar.get_height() * 1.01
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}%',
             ha='center', va='bottom')

# Customize the plot
plt.ylim(0, max(results_percentages.values) * 1.1)  # Add 10% padding above bars

# Remove the box on top and right
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Make the axes white including text and ticks
plt.gca().spines['bottom'].set_color('white')
plt.gca().spines['left'].set_color('white')
plt.gca().tick_params(colors='white')
plt.gca().xaxis.label.set_color('white')
plt.gca().yaxis.label.set_color('white')


plt.show()

NameError: name 'pd' is not defined